## 泰坦尼克号数据分析

本案例基于Kaggle上的Titanic数据集,对此数据集进行初步探索、查看自变量分布、自变量与因变量关系分布、特征处理、建模分析及模型评估等，让大家快速了解Python数据分析的大致流程。

# 目录

* 1.[加载数据](#1.加载数据)  
* 2.[数据的初步探索](#2.数据的初步探索)  
* 3.[汇总及描述性统计](#3.汇总及描述性统计)  
* 4.[变量分布统计](#4.变量分布统计)  
* 5.[探索变量之间的关系](#5.探索变量之间的关系)  
    * 5.1 [探索单个变量与Survived的关系](#5.1探索单个变量与Survived的关系)<br> 
    * 5.2 [探索多个变量与Survived的关系](#5.2探索多个变量与Survived的关系)<br> 
* 6.[特征处理](#6.特征处理) 
    * 6.1 [头衔转换](#6.1头衔转换)<br> 
    * 6.2 [空值处理](#6.2年龄转换)<br> 
* 7.[哑变量编码](#7.哑变量编码)  
* 8.[建模分析](#8.建模分析)  
    * 8.1 [切割训练数据](#8.1切割训练数据)<br> 
    * 8.2 [使用LogisticRegression建模](#8.2使用LogisticRegression建模)<br> 
* 9.[模型评估](#9.模型评估)  
    


# 1.加载数据

In [1]:
import pandas as pd
Titanic = pd.read_csv("./input/Titanic.csv")#训练数据集

FileNotFoundError: File b'./input/Titanic.csv' does not exist

In [2]:
Titanic.shape

NameError: name 'Titanic' is not defined

首先，我们对本案例的数据集进行初步的了解。数据集共有**12**列，目标变量**Survived**表示的是该乘客是否获救,**1**表示获救，**0**表示未获救。其余都是乘客的个人信息，包括：

* **PassengerId => 乘客ID**
* **Pclass => 乘客等级(1/2/3等舱位)**
* **Name => 乘客姓名**
* **Sex => 性别**
* **Age => 年龄**
* **SibSp => 堂兄弟/妹个数**
* **Parch => 父母与小孩个数**
* **Ticket => 船票信息**
* **Fare => 票价**
* **Cabin => 客舱**
* **Embarked => 登船港口(C/Q/S港口)**

# 2.数据的初步探索

In [3]:
#查看数据集的行列数
Titanic.shape

NameError: name 'Titanic' is not defined

In [ ]:
#查看前两行
Titanic.head(2)

# 3.汇总及描述性统计

In [ ]:
#查看数据缺失、数据类型等概况
Titanic.info()

训练数据中总共有**891**名乘客，但是观察后发现有些特征存在缺失值，例如：

* `Age`（年龄）只有**714**名乘客有记录。

* `Cabin`（客舱）只有**204**名乘客是已知的。

In [ ]:
#统计每一列的均值、最大值、最小值、分位数等
Titanic.describe(include='all')

观察上述数据我们可以获得一些简单信息，如：

* `Survived`变量均值告诉我们，约有**38.38%**的人获救了。

* 平均乘客年龄大概是**29.7**岁等。

# 4.变量分布统计

In [4]:
#1.获救情况分布，共891位乘客，仅有300多位乘客幸免于难,占比38%
Titanic['Survived'].value_counts().plot(kind='bar',color='yellow',title='Rescue situation', rot=360)

NameError: name 'Titanic' is not defined

In [ ]:
#2.性别分布，共891位乘客，男性乘客就有577位,占比达64.76%
Titanic['Sex'].value_counts().plot(kind='bar',color='pink',title='Gender distribution', rot=360)

In [ ]:
#3.船舱分布，其中三等舱人数最多，一等舱人口次之。一等舱约占1/4，三等舱约占1/2
Titanic['Pclass'].value_counts().plot(kind='bar',color='green',title='Pclass distribution', rot=360)

前三个探索的变量均为类别变量，接下来探索连续型变量`Age`。

In [ ]:
#4.年龄分布，乘客的年龄都集中在20-40岁之间
Titanic['Age'].plot(kind='hist',color='pink',title='Age distribution')

# 5.探索变量之间的关系

## 5.1探索单个变量与Survived的关系

看过电影《Titanic》的人都会记得，影片中女士与孩子、社会地位较高者优先得到了救助。

这表明`Age`，`Sex`和`PClass`可能是影响生存的关键因素，那么我们将通过图表来展现`Sex`和`Pclass`分别与`Survived`的关系。

首先我们通过分组和聚合函数来实现：

In [ ]:
 Titanic[['Sex', 'Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived',ascending=False)

同样也可以通过透视表的方式来实现：

In [ ]:
sex_pivot = Titanic.pivot_table(index="Sex",values="Survived")
sex_pivot

In [5]:
import matplotlib.pyplot as plt

#构造Sex与Survived均值的条形图
sex_pivot.plot.bar(rot=360)
plt.show()

NameError: name 'sex_pivot' is not defined

女性的幸存比例明显高于男性，现在用同样的方式来看看`Pclass`与`Survived`的关系：

In [ ]:
Titanic[['Pclass', 'Survived']].groupby(['Pclass'],as_index=False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
class_pivot = Titanic.pivot_table(index="Pclass",values="Survived")
class_pivot

In [ ]:
class_pivot.plot.bar(rot=360)
plt.show()

可见，一等舱乘客幸存比例高于二等与三等舱。

同时，我们也可以通过图表来展现`SibSp`(堂兄弟/妹个数)和`Parch`(父母与小孩个数)分别与`Survived`的关系。

In [ ]:
Titanic[['SibSp', 'Survived']].groupby(['SibSp'],as_index=False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
SibSp_pivot = Titanic.pivot_table(index="SibSp",values="Survived")
SibSp_pivot

In [ ]:
SibSp_pivot.plot.bar(rot=360)
plt.show()

我们可以观察到有1-2个堂兄弟姐妹获救的概率最高。

In [ ]:
Titanic[['Parch', 'Survived']].groupby(['Parch'],as_index=False).mean().sort_values(by='Survived',ascending=False)

In [ ]:
Parch_pivot = Titanic.pivot_table(index="Parch",values="Survived")
Parch_pivot

In [ ]:
Parch_pivot.plot.bar(rot=360)
plt.show()

可以看出，父母/孩子有1个或3个幸存率最高。

## 5.2探索多个变量与Survived的关系

In [ ]:
import seaborn as sns
g = sns.FacetGrid(Titanic, col='Survived')
g.map(plt.hist, 'Age', bins=20)

首先探索年龄与获救人数的关系：
+ 在获救的人群中，20-40岁的人占比最多；
+ 在遇难的人群中，18-30岁的人占比最多。

In [ ]:
grid = sns.FacetGrid(Titanic, col='Survived', row='Pclass', height=2.2,aspect=1.6)
grid.map(plt.hist, 'Age', alpha=0.5, bins=20)

探索年龄、舱位与获救人数的关系：<br>
+ 一等舱获救的比例明显高于其他舱位；<br>
+ 一等舱和二等舱的孩子基本上都获救了；
+ ...

In [ ]:
grid = sns.FacetGrid(Titanic, row='Embarked', height=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived','Sex',palette='deep',order=[1,2,3],hue_order=['male','female'])
grid.add_legend()

探索登船港口、舱位、性别与获救比例的关系：<br>
+ 一等舱中从港口Q和港口S登船的女性基本上都获救了；<br>
+ 从C港口登船的男性获救比例很高；
+ ...

同样，我们还可以加上探索登船口，性别、是否幸存与船票费用之间的关系。

In [ ]:
grid = sns.FacetGrid(Titanic, row='Embarked', col='Survived',height=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=0.5, ci=None, order=['female','male'])
grid.add_legend()

明显可以看出：
+ 从S和C港口登船的人中，获救的人群船票明显更贵;
+ ...

# 6.特征处理

## 6.1头衔转换
我们可以看出`Name`列中都有每个人的称呼和头衔，如Mr，Master，Mrs等等，这些不仅代表了性别和身份地位，而且很有可能跟是否获救还有一些关系，所以我们单独把这一个属性摘取出来新成一列。

In [ ]:
Titanic['Title'] = Titanic['Name'].str.extract('([A-Za-z]+)\.',expand=False)
#交叉表
pd.crosstab(Titanic['Title'], Titanic['Sex'])

In [ ]:
def process_age(df):
    df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.',expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', \
                                                 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

In [ ]:
process_age(Titanic)

In [ ]:
#查看头衔提取与生还的关系
Titanic[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
pivot = Titanic.pivot_table(index="Title",values='Survived')
pivot

In [ ]:
pivot.plot.bar(rot=360)
plt.show()

显然，Mr生还的比例最低，Miss和Mrs获救比例最大。

## 6.2年龄转换

`Sex`和`PClass`变量属于类别变量，它们的值代表了几个不同的类别，例如，乘客是男性还是女性。

通过`Series.describe()`查看`Age`列概况。

In [ ]:
Titanic["Age"].describe()

乘客的年龄分布在**0.42-80.0**之间，训练集共814条数据，年龄字段共有714个值，这表明存在部分缺失值。

由于年龄字段是一个连续变量，我们可通过直方图来查看其分布情况。为了更好地查看不同年龄段与是否生还之间的关系，我们可以创建两个直方图来探索分析。

In [ ]:
#使用布尔索引分别得到获救与未获救DataFrame
survived = Titanic[Titanic["Survived"] == 1]
died = Titanic[Titanic["Survived"] == 0]

#建立直方图查看不同年龄的获救对比情况
survived["Age"].plot.hist(alpha=0.5,color='red',bins=50)
died["Age"].plot.hist(alpha=0.5,color='blue',bins=50)
plt.legend(['Survived','Died'])
plt.show()

可以看到，在一些年龄范围内，比如**0-20**岁之间，更多的乘客幸存下来（红条高于蓝条）。那么接下来尝试使用`pandas.cut()`函数来将年龄字段进行分段，转换成类别变量。具体实现如下：

我们将创建一个函数：

1. 使用`pandas.fillna()`方法用-0.5填充所有缺失的值。

2. 将`Age`变量切成六段：

    * **Missing，从-1到0**
    * **Infant，从0到5**
    * **Child，从5岁到12岁**
    * **Teenager，从12岁到18岁**
    * **Young Adult，从18岁到35岁**
    * **Adult，从35到60**
    * **Senior，从60到100**
    
下图很好的展示函数如何转换数据的过程：

<img src="http://cookdata.cn/media/note_images/微信图片_20180419225815_1524149902345_1f8b.jpg"  Width="400px">

In [ ]:
#定义年龄分段处理函数
def process_age(df,cut_points,label_names):
    df["Age"] = df["Age"].fillna(-0.5)#使用-0.5填充所有缺失值
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)#新建Age_categories用来存放分段后的年龄信息
    return df

cut_points = [-1,0,5,12,18,35,60,100] #自定义的年龄分段区间
label_names = ["Missing","Infant","Child","Teenager","Young Adult","Adult","Senior"] #各分段区间所属标签列表

In [ ]:
#在训练集上调用process_age函数
Titanic = process_age(Titanic,cut_points,label_names)

In [ ]:
#查看分段后的年龄与生还的关系
pivot = Titanic.pivot_table(index="Age_categories",values='Survived')
pivot.plot.bar(rot=360)
plt.show()

从上图可以看出**Infant**的幸存数是最多的。

# 7.哑变量编码

虽然每个乘客的`Pclass`类别存在某种有序的关系，但每个类别之间的关系与数字1、2和3之间的关系并不相同。<br>
例如，类别2不是类别1的两倍，类别3级不是类别1的三倍。

我们可以将`Pclass`转为哑变量编码来解决这个问题。同理可以对类别变量`Sex`与`Age_categories`做相同处理。

<img src='http://cookdata.cn/media/note_images/微信图片_20180419232330_1524151408055_1f8b.jpg' width='400px'>

In [ ]:
#创建create_dummies函数
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name) #prefix参数为变量转换后列名的前缀
    df = pd.concat([df,dummies],axis=1)
    return df

#遍历训练集、测试集的Pclass、Sex、Age_categories、Title变量，对其进行哑变量的转换
for column in ["Pclass","Sex","Age_categories","Title"]:
    Titanic = create_dummies(Titanic,column)

# 8.建模分析

## 8.1切割训练数据

我们可以在训练集上拟合和预测，但这样训练出来的模型很有可能会过度拟合，因为它会在已经训练过的数据表现良好，却在一些新的数据集上表现很差。

因此，我们可以将训练集分成两部分：

* 一部分用来训练模型（通常为观测值的80%）

* 一部分用来预测（通常为观测值的20％）

可通过`scikit-learn`中的`model_selection.train_test_split()`函数进行数据切割。`train_test_split()`包含两个参数`X`和`y`，它们包含其中`X`指的是特征变量，`y`指的是目标变量。函数返回四个对象分别是：**train_X，train_y，test_X，test_y**：

<img src='http://cookdata.cn/media/note_images/微信图片_20180420085749_1524193517590_1f8b.jpg' width='500px'>

In [ ]:
#将要放入模型进行训练的特征变量
columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior','Title_Master','Title_Miss','Title_Mr','Title_Mrs','Title_Rare']

all_X = Titanic[columns]

#训练集的目标变量
all_y = Titanic['Survived']

#train_test_split()函数实现训练集分割，test_size为分割比例，random_state随机种子是为了保证重复试验的时候，保证得到一组一样的随机数
train_X, test_X, train_y, test_y = train_test_split(all_X, all_y, test_size=0.20,random_state=0)

## 8.2使用LogisticRegression建模

现在数据已经准备好了，来训练一个LogisticRegression模型。
具体步骤如下：

In [ ]:
#导入sklearn的LogisticRegression模型
from sklearn.linear_model import LogisticRegression

#创建LogisticRegression对象：
lr = LogisticRegression()

最后，我们使用`LogisticRegression.fit()`方法来训练我们的模型。`.fit()`方法包含两个参数：`X`和`y`。 `X`是一个特征变量的二维数组（如DataFrame），`y`是一个目标变量的一维数组（如Series）。

In [ ]:
lr.fit(Titanic[columns], Titanic["Survived"])

# 9.模型评估

In [ ]:
lr = LogisticRegression()
lr.fit(train_X, train_y)
predictions = lr.predict(test_X)

模型拟合并预测后，我们肯定会关心预测结果的准确性。scikit-learn中的metrics.accuracy_score()函数就可实现。该函数中的两个参数分别是y_true(实际值)和y_pred(预测值)，返回结果是准确性分数。具体如下：

In [6]:
from sklearn.metrics import accuracy_score
#计算预测准确性函数
accuracy = accuracy_score(test_y, predictions)

accuracy

NameError: name 'test_y' is not defined

在20％测试集数据上进行预测时，模型的准确性得分为79.0％。